# Web Scrapping

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from time import sleep

# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.common.exceptions import TimeoutException
# from splinter import Browser

# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=True)

## Scraping ATP Top 100 Player Profiles

In [6]:
def parse_soup(sp):
    label_temp = []
    value = []

    # Players name and appending to value
    first_name = sp.find('div', class_='first-name').text
    last_name = sp.find('div', class_='last-name').text
    player_name = first_name + ' ' + last_name
    value.append(player_name)

    # This is pulling the Rank label and if it is for singles or doubles
    data_label_rank = sp.find('div', class_='data-label-text').text.strip()
    data_label_rank = re.sub('\r\n', ' ', data_label_rank)
    label_temp.append((re.sub(' +', ' ', data_label_rank)))

    # This is pull the value for the rank label
    data_number = sp.find('div', class_='data-number').text.strip()
    value.append(data_number)
    # rank = data_number  # Variable for class

    # This is pulling the labels Age, Turned Pro, Weight, and Height.
    for table_big_label in sp.find_all('div', class_='table-big-label'):
        label_temp.append(table_big_label.text)

    # This is pulling the values for the labels above

    for table_big_value in sp.find_all('div', class_='table-big-value'):
        x = table_big_value.text.strip()
        bad = "\n\r\n"
        for r in bad:
            x = x.replace(r, "")
        value.append(re.sub(' +', ' ', x))

    # This is pulling labels for Birthplace, Plays, Coach
    for table_label in sp.find_all('div', class_='table-label'):
        label_temp.append(table_label.text.strip())

    # This is pulling labels for Birthplace, Plays, Coach
    for table_value in sp.find_all('div', class_='table-value'):
        value.append(table_value.text.strip())

    return value

def player(value):

    # This loop is taking the string of variables and assigning them to their respected values
    label = ['fullname', 'rank', 'age', 'turned_pro', 'weight', 'height', 'birthplace', 'plays', 'coach']
    n = 0
    for name in label:
        globals()[name] = value[n]
        n += 1
    return fullname, rank, age, turned_pro, weight, height, birthplace, plays, coach

In [8]:
value_list = []  # Empty list to insert each player into in
html_text_rank = requests.get('https://www.atptour.com/en/rankings/singles').text
soup_rank = BeautifulSoup(html_text_rank, 'lxml')

url_link = []
html_1 = 'https://www.atptour.com'  # Needed this to add the link a pulled in from singles rank page

# This for loop is grabbing the links to the players overview if the link doesn't contain overview it passes
for a in soup_rank.find_all('a', href=True):
    link = a['href']
    if 'overview' in link:
        url = html_1 + link
        url_link.append(url)
    else:
        pass
print('all links grabbed')
# this for loop runs the functions above to grab the info from overview for each player and appends to value_list
for i in url_link:
    html_text = requests.get(i).text
    soup = BeautifulSoup(html_text, 'lxml')
    player(parse_soup(soup))
    value_list.append((fullname, int(rank), age, turned_pro, weight, height, birthplace, plays, coach))
    print(fullname, 'added to list')
# This is creating the dataframe
df = pd.DataFrame(value_list, columns=['fullname', 'rank', 'age', 'turned_pro', 'weight', 'height', 'birthplace', 'plays', 'coach'])
df.head()    

all links grabbed
Carlos Alcaraz added to list
Rafael Nadal added to list
Casper Ruud added to list
Stefanos Tsitsipas added to list
Novak Djokovic added to list
Andrey Rublev added to list
Felix Auger-Aliassime added to list
Daniil Medvedev added to list
Taylor Fritz added to list
Holger Rune added to list
Hubert Hurkacz added to list
Cameron Norrie added to list
Alexander Zverev added to list
Matteo Berrettini added to list
Pablo Carreno Busta added to list
Jannik Sinner added to list
Frances Tiafoe added to list
Marin Cilic added to list
Lorenzo Musetti added to list
Karen Khachanov added to list
Nick Kyrgios added to list
Denis Shapovalov added to list
Borna Coric added to list
Alex de Minaur added to list
Roberto Bautista Agut added to list
Diego Schwartzman added to list
Miomir Kecmanovic added to list
Grigor Dimitrov added to list
Francisco Cerundolo added to list
Daniel Evans added to list
Sebastian Korda added to list
Alejandro Davidovich Fokina added to list
Yoshihito Nishiok

,fullname,rank,age,turned_pro,weight,height,birthplace,plays,coach
0,Carlos Alcaraz,1,19 (2003.05.05),2018,163lbs(74kg),"6'0""(183cm)","El Palmar, Murcia, Spain","Right-Handed, Two-Handed Backhand",Juan Carlos Ferrero
1,Rafael Nadal,2,36 (1986.06.03),2001,187lbs(85kg),"6'1""(185cm)","Manacor, Mallorca, Spain","Left-Handed, Two-Handed Backhand","Carlos Moya, Marc Lopez, Gustavo Marcaccio"
2,Casper Ruud,3,24 (1998.12.22),2015,170lbs(77kg),"6'0""(183cm)","Oslo, Norway","Right-Handed, Two-Handed Backhand",Christian Ruud
3,Stefanos Tsitsipas,4,24 (1998.08.12),2016,198lbs(90kg),"6'4""(193cm)","Athens, Greece","Right-Handed, One-Handed Backhand",Apostolos Tsitsipas
4,Novak Djokovic,5,35 (1987.05.22),2003,170lbs(77kg),"6'2""(188cm)","Belgrade, Serbia","Right-Handed, Two-Handed Backhand",Goran Ivanisevic


## Scraping Livescore.in for point by point data

In [22]:
temp = []
condition = "/tennis/atp-singles/"

url = "https://www.livescore.in/tennis/atp-singles/"

# get html content
r = requests.get(url)
html_content = r.text
# print(html_content)
# parse the html using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# find class="lmc__template"
for i in soup.find_all("a", href=True):
    link = i['href']
    if condition in link:
        temp.append(link)
    else:
        pass
print(temp)

['/tennis/atp-singles/', '/tennis/atp-singles/australian-open/', '/tennis/atp-singles/', '/tennis/atp-singles/adelaide/', '/tennis/atp-singles/amersfoort/', '/tennis/atp-singles/antalya/', '/tennis/atp-singles/antwerp/', '/tennis/atp-singles/atp-cup/', '/tennis/atp-singles/auckland/', '/tennis/atp-singles/australian-open/', '/tennis/atp-singles/bangkok/', '/tennis/atp-singles/barcelona/', '/tennis/atp-singles/basel/', '/tennis/atp-singles/bastad/', '/tennis/atp-singles/beijing/', '/tennis/atp-singles/belgrade/', '/tennis/atp-singles/bogota/', '/tennis/atp-singles/brisbane/', '/tennis/atp-singles/budapest/', '/tennis/atp-singles/buenos-aires/', '/tennis/atp-singles/casablanca/', '/tennis/atp-singles/chengdu/', '/tennis/atp-singles/chennai/', '/tennis/atp-singles/copenhagen/', '/tennis/atp-singles/cordoba/', '/tennis/atp-singles/costa-do-sauipe/', '/tennis/atp-singles/davis-cup-world-group/', '/tennis/atp-singles/doha/', '/tennis/atp-singles/dubai/', '/tennis/atp-singles/dusseldorf/', '/

In [34]:
browser.quit()

In [50]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from time import sleep

In [51]:
base_url = 'https://www.livescore.in/tennis/atp-singles/'

# invoke the webdriver
# change the executable path based on where you have saved your chromedriver
options = selenium.webdriver.ChromeOptions()
#options.add_argument('--headless')
#could also do options.headless = True
options.add_argument('--window-size=1920x1080')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
# go to the webpage that we want to scrape
driver.get(base_url)
# wait for the page to load
delay = 3 # seconds
try:
    myElem = WebDriverWait(driver, delay)
    print("Page is ready!")
except TimeoutException:
    print("Loading took too much time!")


/var/folders/6q/s2d_6jyn2fd12h8x6pvt0d_00000gp/T/ipykernel_2398/1606498032.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


Page is ready!


In [52]:
driver.close()